<a href="https://colab.research.google.com/github/K02D/HLTV-Rating-Breakdown/blob/main/HLTV_Rating_Breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!pip install eli5

In [ ]:
from google.colab import files
files.upload()
# upload kaggle.json file obtained from Kaggle by going to Account -> Create New API Token

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Get dataset from kaggle
!kaggle datasets download "mateusdmachado/csgo-professional-matches"

In [ ]:
!unzip csgo-professional-matches.zip players.csv

In [11]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance

data = pd.read_csv("players.csv")

# Remove records where the old rating was used
data_updated = data.dropna(axis=0)

# Set prediction target
y = data_updated.rating

# Set input features
features = ['kills', 'assists', 'deaths', 'flash_assists', 'kast', 'adr', 'kddiff']
X = data_updated[features]

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

# Use gradient boosting to fit a model
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(X_train, y_train, early_stopping_rounds=5,
          eval_set=[(X_val, y_val)],
          verbose=False)

# Show relative importance of each feature in the model
perm = PermutationImportance(model, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm, feature_names=X_val.columns.tolist())

[17:03:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.6732 ± 0.0197,kddiff
0.2120 ± 0.0070,adr
0.0455 ± 0.0019,kast
0.0046 ± 0.0004,kills
0.0037 ± 0.0004,deaths
0.0017 ± 0.0002,assists
0.0003 ± 0.0001,flash_assists
